> this notebook contain the solution of the exercises from the video "The spelled-out intro to language modeling: building makemore" by [Andrej Karpathy](https://youtu.be/PaCmpygFfXo?si=-Rvr6JoZFtCKS21x) and where he build the bigram model using simple counting(sampling over prob distribution) and neural net. (5 exercises)


> **E01: train a `trigram language model`, i.e. take `two characters as an input to predict the 3rd one`. Feel free to use either `counting` or a `neural net`. Evaluate the loss; Did it improve over a bigram model?**

if you don't have any idea about N-grams model then i highly recommend you to go through this [stanford-notes](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

In [210]:
# data 

data = open('names.txt', 'r').read().splitlines()
len(data)

32033

In [212]:
data[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [213]:
# checking the max and min lenght of the words in data
out = []
for w in range(len(data)):
    out.append(len(data[w]))
print(max(out)," ", min(out))

15   2


In [216]:
data[0]

'emma'

In [219]:
data[0][0],data[0][1:2], data[0][2:3] 

('e', 'm', 'm')

In [231]:
t = {}

for w in data:
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 , ch3 in zip(chs, chs[1:], chs[2:] ):
        tigram = (ch1, ch2, ch3)
        t[tigram] = t.get(tigram,0) + 1
        
        

In [1]:
# sorted(t.items(), key = lambda kv : -kv[1])

In [238]:
chars = sorted(set(''.join(data)))
stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27
stoi

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 '<S>': 26,
 '<E>': 27}

In [249]:
# removing the <E> and <S>.

chars = sorted(set(''.join(data)))
stoi = {s:i for i,s in enumerate(chars)}
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [250]:
# we want 3d array (cuause x1, x2 -> y)
import torch 
N = torch.zeros((27,27,27),dtype=torch.int32)
for w in data:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 , ch3 in zip(chs, chs[1:], chs[2:] ):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[ix1,ix2,ix3] += 1

### sampling from the model 

In [257]:
P = (N+1).float()
P = P / P.sum(2, keepdims=True) # we need to get probablibility .. so that's why normalizing the two dim.

In [258]:
P[1, 1].sum() # sum of porb should be 1 then we are right in what we do above 

tensor(1.0000)

In [280]:
# sampling from new probs

g = torch.Generator().manual_seed(2147483647)

for i in range(10):
  out = ['.', '.']
  while True:
    p = P[stoi[out[-2]], stoi[out[-1]]] # cause we need last two char to predict new 
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

..ce.
..za.
..zogh.
..uriana.
..kaydnevonimittain.
..luwak.
..ka.
..da.
..samiyah.
..javer.


In [283]:
# loss 

log_likehood = 0.0
n = 0
for w in data:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    n+=1
    logprob = torch.log(P[ix1, ix2, ix3])
    log_likehood += logprob

print(f'{log_likehood:.4f}')
nll = -log_likehood

print(nll / n) # well loss is improved from the bigram model.

-410414.9688
tensor(2.0927)


### with neural network 

In [303]:
xs, ys = [], []
for w in data:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1, ix2))# two input again  
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(ys.shape[0])



196113


In [304]:
# init weights 
g = torch.Generator().manual_seed(2147483647)
W =  torch.randn((27*2, 27), generator=g, requires_grad=True) # 27*2 two diff weight for two inputs ix1 and ix2 

In [292]:
import torch 
from torch import nn
import torch.nn.functional as F

for k in range(1000):
  xenc = F.one_hot(xs, num_classes=27).float() 
  
 
  logits = xenc.view(-1, 27*2) @ W 
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() 

  W.grad = None
  loss.backward()

  W.data += -3 * W.grad
print(loss.item())

2.283792018890381


> **E02: split up the dataset randomly into `80% train set, 10% dev set, 10% test set`. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?**

In [294]:
import random 
import math

random.shuffle(data)

lenght = len(data)

train_data = data[:math.floor(lenght * 0.8)]
dev_data = data[math.floor(lenght*0.8):math.floor(lenght*0.9)]
test_data = data[math.floor(lenght*0.9):]

In [305]:
xs, ys = [], []
for w in train_data:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1, ix2)) 
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(ys.shape[0])

g = torch.Generator().manual_seed(2147483647)
W =  torch.randn((27*2, 27), generator=g, requires_grad=True) 

156850


In [306]:
import torch 
from torch import nn
import torch.nn.functional as F

for k in range(1000):
  xenc = F.one_hot(xs, num_classes=27).float() 
  
 
  logits = xenc.view(-1, 27*2) @ W 
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() 

  W.grad = None
  loss.backward()

  W.data += -3 * W.grad
print(loss.item())

2.281895637512207


#### loss on test_data

In [312]:

loss = []
for w in test_data:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
      
    xenc = F.one_hot(torch.tensor((ix1, ix2)), num_classes=27).float()
    logits = xenc.view(-1, 27*2) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss.append(-probs[0, ix3].log())
print(torch.tensor(loss).mean()) 



tensor(2.2946)


#### loss on dev_data

In [314]:
loss = []
for w in dev_data:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
      
    xenc = F.one_hot(torch.tensor((ix1, ix2)), num_classes=27).float()
    logits = xenc.view(-1, 27*2) @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    loss.append(-probs[0, ix3].log())
print(torch.tensor(loss).mean()) 

tensor(4.2705)


> **E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?**

In [321]:
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

for i, alpha in enumerate(alphas):
    W =  torch.randn((27*2, 27), generator=g, requires_grad=True)
    for k in range(1000):
      xenc = F.one_hot(xs, num_classes=27).float()
      logits = xenc.view(-1, 27*2) @ W
      counts = logits.exp()
      probs = counts / counts.sum(1, keepdims=True)
      loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() + alpha*(W**2).mean()
      W.grad = None
      loss.backward()
      W.data += -3 * W.grad
    print(f"alpha: {alpha} : {loss.item()}")

    loss = []
    for w in dev_data:
      chs = ['.'] + list(w) + ['.']
      for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
          
        xenc = F.one_hot(torch.tensor((ix1, ix2)), num_classes=27).float()
        logits = xenc.view(-1, 27*2) @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdims=True)
        loss.append(-probs[0, ix3].log())
    print(f"alpha: {alpha} : {torch.tensor(loss).mean()}")
    print()

alpha: 1e-05 : 2.2819204330444336
alpha: 1e-05 : 2.289590835571289

alpha: 0.0001 : 2.281435966491699
alpha: 0.0001 : 2.2894060611724854

alpha: 0.001 : 2.2805747985839844
alpha: 0.001 : 2.2872159481048584

alpha: 0.01 : 2.288482904434204
alpha: 0.01 : 2.2868690490722656

alpha: 0.1 : 2.3399317264556885
alpha: 0.1 : 2.2893760204315186



##### result is improving .. it is saving model from on getting overfit on train_data.

> **E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?**

In [322]:

xs, ys = [], []
for w in data:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append((ix1, ix2))  #
        ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)



g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27 * 27, 27), generator=g, requires_grad=True)  

# train loop
for k in range(1000):
   
    indices = xs[:, 0] * 27 + xs[:, 1]  # to remove f.one_hot

    # lookup rows of w using combined indices
    logits = W[indices]  # logits has shape [examplesCount, 27]

    # cal probabilities
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)

    
    loss = -probs[torch.arange(ys.shape[0]), ys].log().mean()

    
    W.grad = None
    loss.backward()

    
    W.data += -3 * W.grad # we chaneg this '-3'

print(loss.item())


2.5359888076782227


> **E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?**

In [327]:
xs, ys = [], []
for w in data:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append((ix1, ix2))  #
        ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)



g = torch.Generator().manual_seed(2147483647)
W = torch.randn((54, 27), generator=g, requires_grad=True)  


for k in range(1000):
    
    xenc = F.one_hot(xs, num_classes=27).float() 
    logits = xenc.view(-1, 54) @ W  


    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)

  
    loss = -probs[torch.arange(ys.shape[0]), ys].log().mean() + 0.01 * (W ** 2).mean()

   
    cross_entropy_loss = F.cross_entropy(logits, ys) + 0.01 * (W ** 2).mean()

   
    W.grad = None
    loss.backward()

    
    W.data += -3 * W.grad

print(loss.item())
print(cross_entropy_loss.item())

2.291670322418213
2.291670322418213


> bye !! 🐳